<a href="https://colab.research.google.com/github/g-roger/natural-language-process/blob/main/Aula_5_6_IA_PLN_Exerc%C3%ADcios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercício!

O código abaixo vai montar o Google Drive como um disco e torna-lo acessível para salvarmos os arquivos no drive! 

**É necessário apenas para quem usar o Google Colab**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


A classe abaixo nos ajudará a acompanhar o progresso do download.

##**Exercício**

1.   Crie uma função que substitua uma ou mais palavras de um texto um uma palavra similar com base na sua classe gramatical.
2.   Extenda essa função para considerar uma substituição positiva ou negativa.

In [ ]:
!pip install scipy==1.2.0
!pip install gensim==2.0.0
# Obs: pode ser necessário reiniciar o Runtime do colab

!pip install spacy
!python -m spacy download pt
#!python -m spacy download pt_core_news_sm

In [ ]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import zipfile

class DLProgress(tqdm):
  last_block = 0

  def hook(self, block_num=1, block_size=1, total_size=None):
    self.total = total_size
    self.update((block_num - self.last_block) * block_size)
    self.last_block = block_num

In [ ]:
url = 'http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s300.zip'
#url = 'https://dados-ml-pln.s3-sa-east-1.amazonaws.com/cbow_s300.zip'

tar_gz_path = '/content/gdrive/My Drive/FIAP/NLP/cbow_s300.zip'

if not isfile(tar_gz_path):
  with DLProgress(unit='B', unit_scale=True, miniters=1, desc='Word2Vec Model') as pbar:
    urlretrieve(
        url,
        tar_gz_path,
        pbar.hook)

if not isfile('/content/gdrive/My Drive/FIAP/NLP/cbow_s300.txt'):     
  zip_ref = zipfile.ZipFile(tar_gz_path, 'r')
  zip_ref.extractall('/content/gdrive/My Drive/FIAP/NLP/')
  zip_ref.close()

In [ ]:
!ls -all -h /content/gdrive/My\ Drive/FIAP/NLP/cbow_s300.txt

-rw------- 1 root root 2.5G Nov 10 16:43 '/content/gdrive/My Drive/FIAP/NLP/cbow_s300.txt'


In [ ]:
# outro exemplo de baixar o arquivo e descompactar, só não esqueça de alterar o diretório apontando o aquivo para a raiz do colabo ao carregar o modelo
'''
!wget 'http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s300.zip'
!ls -la # veja o nome do arquivo compactado salvo no download
!unzip 'download.php?file=embeddings%2Fword2vec%2Fcbow_s300.zip' # subistitua com nome do arquivo
!ls -la
'''

In [ ]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('/content/gdrive/My Drive/FIAP/NLP/cbow_s300.txt')

Slow version of gensim.models.doc2vec is being used


In [ ]:
model

Exemplo 1:

In [ ]:
import spacy
pt = spacy.load('pt')

In [ ]:
texto = 'Você é baixo'

for token in pt(texto):
  print(token.text, ' - ', token.pos_)

Você  -  PRON
é  -  VERB
baixo  -  ADJ


In [ ]:
texto = 'Você é baixo'
pt(texto)[2].pos_

'ADJ'

In [ ]:
 model.most_similar_cosmul(positive=['baixo'],)

[('alto', 0.85185706615448),
 ('elevado', 0.7905384302139282),
 ('baixíssimo', 0.7482064366340637),
 ('altíssimo', 0.7426224946975708),
 ('elevadíssimo', 0.7309994697570801),
 ('reduzido', 0.7289093732833862),
 ('fraco', 0.7261129021644592),
 ('ele-vado', 0.7119198441505432),
 ('eleva-do', 0.7061087489128113),
 ('bai-xo', 0.6976011991500854)]

In [ ]:
import spacy

#pt = spacy.load('pt_core_news_sm')
pt = spacy.load('pt')

def trocar_por_similar(frase, pos):
  doc = pt(frase)
  
  nova_frase = []
  for token in doc:
    if token.pos_ == pos:
      similares = model.most_similar_cosmul(positive=[token.text] )
      for similar, _ in similares:
        if similar != token.text:
          break
      nova_frase.append(similar)
    else:
      nova_frase.append(token.text)
  return ' '.join(nova_frase)

In [ ]:
pt('Você é baixo')[2].pos_

'ADJ'

In [ ]:
print(trocar_por_similar('Você é baixo', 'ADJ'))

Você é alto


Exemplo 2:

In [ ]:
import spacy

#pt = spacy.load('pt_core_news_sm')
pt = spacy.load('pt')

def trocar_por_similar(frase, pos, sentido):
  doc = pt(frase)
  
  nova_frase = []
  for token in doc:
    if token.pos_ == pos:
      similares = model.most_similar_cosmul(positive=[token.text], negative=[sentido] )
      for similar, _ in similares:
        if similar != token.text:
          break
      nova_frase.append(similar)
    else:
      nova_frase.append(token.text)
  return ' '.join(nova_frase)

In [ ]:
pt('Você é muito alto')[3].pos_

'ADJ'

In [ ]:
print(trocar_por_similar('Você é muito baixo', 'ADJ', 'magro'))

Você é muito altíssimo
